In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define the corrected order of EEG electrode labels
corrected_electrode_order = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 
                             'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']

def calculate_spearman_correlation(preprocessed_file):
    df = pd.read_csv(preprocessed_file)
    grouped = df.groupby('Patient_ID')
    spearman_results = {}

    for patient_id, group in grouped:
        # Filter only the EEG columns in the corrected order
        eeg_data = group[corrected_electrode_order]
        spearman_corr = eeg_data.corr(method='spearman')
        spearman_results[patient_id] = spearman_corr
    
    return spearman_results

def plot_spearman_heatmap(spearman_results, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    for patient_id, corr_matrix in spearman_results.items():
        plt.figure(figsize=(12, 10))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True, 
                    cbar_kws={"shrink": .8}, linewidths=0.5, fmt=".2f")
        plt.title(f'Spearman Correlation Heatmap for Patient ID: {patient_id}', fontsize=16)
        plt.xlabel('EEG Channels', fontsize=14)
        plt.ylabel('EEG Channels', fontsize=14)
        plt.xticks(rotation=45)
        plt.yticks(rotation=45)
        plt.tight_layout()
        
        plt.savefig(os.path.join(output_dir, f'spearman_heatmap_patient_{patient_id}.png'))
        plt.close()



In [2]:
# Example usage
preprocessed_file = 'preprocessed_eeg_data.csv'  # Update this to your preprocessed CSV file path
output_dir = 'Spearman'  # Update this to your desired output directory
os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
spearman_results = calculate_spearman_correlation(preprocessed_file)
plot_spearman_heatmap(spearman_results, output_dir)
